In [1]:
%load_ext jupyter_black
import numpy as np
import torch
from scipy.stats.qmc import Halton
from collections import deque
import matplotlib.pyplot as plt

seed = 42
g_cuda = torch.Generator(device="cuda")
g_cuda.manual_seed(42)
sampler = Halton(d=6, scramble=True)

parameter_space = dict(
    spot=(30, 70),
    path_integral=(25, 150),
    ttm=(0.2, 1),
    t=(0, 0.8),
    vol=(0.1, 0.5),
    r=(0, 0.1),
)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

The black scholes model for the spot price under risk neutral measure is:
$$dS_t = rS_t\,dt + \sigma S_t\,dW(t)$$
Which is equivalent to:
$$d\log(S_t) = \left(r - \frac{\sigma^2}{2}\right)\,dt + \sigma\,dW(t)$$
The corresponding Euler-Maruyama scheme for a discrete grid $\{t_k = \frac{kT}{n},\; k\in\{0,...,n\}\}$ is:
$$d\log(S_{t_{k+1}})  = d\log(S_{t_k}) + \left(r - \frac{\sigma^2}{2}\right)\frac{T}{n} + \sigma \sqrt\frac{T}{n} Z_{k+1}$$
Where $Z$ are i.i.d standard normal variables.

We want to compute the following price :
$$F(t, T, S_t, I_t, r, \sigma) = e^{-r(T-t)}\mathbb E\left[\left(S_T - I_T\right)^+\Big|\,S_t, \,I_t\right]$$
$$\text{Where :}\quad I_t = \frac{1}{t}\int_{0}^{t} S_u\,du$$

In [3]:
def asian_option(
    n_paths: int,
    n_steps: int,
    spot: float,
    path_integral: float,
    t: float,
    dt: float,
    r: float,
    sigma: float,
) -> torch.Tensor:
    """Simulate n_paths of asian option payoffs
    Args:
        - n_paths: number of simulations
        - n_steps: number of steps in the euler scheme
        - spot: initial value of the asset
        - path_integral: mean of the value of the asset at time t
        - t: time of pricing
        - dt: square root of time step
        - r: interest rate
        - sigma: volatility
    Returns:
        - torch.Tensor: sample payoffs
    """
    T = t + dt * dt * n_steps
    sample = torch.normal(
        mean=torch.zeros(n_paths, n_steps).to(device), std=1, generator=g_cuda
    )
    sample = (r - 0.5 * sigma**2) * dt * dt + dt * sigma * sample
    sample = torch.concatenate(
        (
            spot * torch.ones((n_paths, 1)).to(device),
            spot * torch.exp(sample.cumsum(axis=1)),
        ),
        axis=1,
    )
    sample = torch.exp(-r * dt * dt * n_steps) * torch.maximum(
        sample[:, -1]
        - t * path_integral / T
        - dt * dt * n_steps * sample.mean(axis=1) / T,
        torch.zeros(1).to(device),
    )
    # We use the same name variable sample to free up gpu memory of unecessary data
    return sample

In [4]:
NTPB = 1024
NB = 1024
# n = NB * NTPB
n = 1000
T = 1.0
S_0 = 50.0
sigma = 0.2
r = 0.1
N = 100
t = 0.2
I = 60.0

In [5]:
# Test
asian_option(int(1e6), N, S_0, I, t, torch.sqrt(torch.tensor(T / N)), r, sigma).mean()

tensor(3.1886, device='cuda:0')

In [6]:
# vectorizing the payoffs sampler for batch operations
vmap_asian_option = torch.vmap(
    asian_option,
    in_dims=(None, None, 0, 0, 0, 0, 0, 0),
    out_dims=0,
    randomness="different",
)

We want to learn $F(t, T, S_t, I_t, r, \sigma)$ for a predefined grid of parameters via an MLP, this translates to finding:
$$\theta^* \in \argmin_{\theta\in\Theta}\mathbb E_{x\sim D}\left[(F(x) - T_\theta(x))^2\right]$$
Where $x = (t, T, S_t, I_t, r, \sigma)$ and $D$ is a prior distribution over the parameter space. Since we are attempting to learn an expectation, we can rewrite the problem as:
$$\theta^* \in \argmin_{\theta\in\Theta}\mathbb E_{x\sim D}\left[\mathbb E_{(S_T, I_T)}\left[ (S_T - I_T)^+ - T_\theta(x))^2 \Big | x\right]\right]$$

We can thus train our network on payoffs and test it agaisnt MC estimations of the price.

### Generating training data

In [7]:
# Sampling parameters from a grid
n = 1000
N = 100
nb_samples = int(1e6)
sample_params = sampler.random(n=nb_samples)
sample_params = np.array(
    [
        parameter_space["spot"][1] - parameter_space["spot"][0],
        parameter_space["path_integral"][1] - parameter_space["path_integral"][0],
        parameter_space["t"][1] - parameter_space["t"][0],
        parameter_space["ttm"][1] - parameter_space["ttm"][0],
        parameter_space["r"][1] - parameter_space["r"][0],
        parameter_space["vol"][1] - parameter_space["vol"][0],
    ]
) * sample_params + np.array(
    [
        parameter_space["spot"][0],
        parameter_space["path_integral"][0],
        parameter_space["t"][0],
        parameter_space["ttm"][0],
        parameter_space["r"][0],
        parameter_space["vol"][0],
    ]
)

sample_params[:, 1] = (sample_params[:, 3] < 0.05) * sample_params[:, 0] + (
    sample_params[:, 3] >= 0.05
) * sample_params[:, 0] * np.random.uniform(low=0.5, high=2, size=len(sample_params))

In [8]:
# Seperate parameters into batches
sample_params = torch.tensor(sample_params).to(device)
data_loader = torch.utils.data.DataLoader(sample_params, batch_size=1024)

In [9]:
# Compute n payoffs per sample param
results = deque()
for sample in data_loader:
    sample_payoffs = vmap_asian_option(
        n,
        N,
        sample[:, 0],
        sample[:, 1],
        sample[:, 2],
        torch.sqrt(sample[:, 3] / N),
        sample[:, 4],
        sample[:, 5],
    ).mean(axis=1)
    sample_payoffs = sample_payoffs.to("cpu")
    results.append(sample_payoffs)

In [10]:
# Saving data
X_train = sample_params.to("cpu").numpy()
with open("X_train.npy", "wb") as f:
    np.save(f, X_train)

In [11]:
with open("Y_train_averaged.npy", "wb") as f:
    np.save(f, torch.cat(list(results), dim=0).numpy(), allow_pickle=True)

### Generating validation data

In [12]:
torch.cuda.empty_cache()

In [13]:
n = NB * NTPB
N = 100
nb_samples = int(1e4)
sample_params = sampler.random(n=nb_samples)
sample_params = np.array(
    [
        parameter_space["spot"][1] - parameter_space["spot"][0],
        parameter_space["path_integral"][1] - parameter_space["path_integral"][0],
        parameter_space["t"][1] - parameter_space["t"][0],
        parameter_space["ttm"][1] - parameter_space["ttm"][0],
        parameter_space["r"][1] - parameter_space["r"][0],
        parameter_space["vol"][1] - parameter_space["vol"][0],
    ]
) * sample_params + np.array(
    [
        parameter_space["spot"][0],
        parameter_space["path_integral"][0],
        parameter_space["t"][0],
        parameter_space["ttm"][0],
        parameter_space["r"][0],
        parameter_space["vol"][0],
    ]
)

sample_params[:, 1] = (sample_params[:, 3] < 0.05) * sample_params[:, 0] + (
    sample_params[:, 3] >= 0.05
) * sample_params[:, 0] * np.random.uniform(low=0.5, high=2, size=len(sample_params))

In [14]:
# Seperate parameters into batches
sample_params = torch.tensor(sample_params).to(device)
data_loader = torch.utils.data.DataLoader(sample_params, batch_size=10)

In [15]:
# Compute n payoffs per sample param and retrieve mc estimator of price
results = deque()
for sample in data_loader:
    sample_payoffs = vmap_asian_option(
        n,
        N,
        sample[:, 0],
        sample[:, 1],
        sample[:, 2],
        torch.sqrt(sample[:, 3] / N),
        sample[:, 4],
        sample[:, 5],
    ).mean(axis=1)
    sample_payoffs = sample_payoffs.to("cpu")
    results.append(sample_payoffs)

In [16]:
# Saving data
X_valid = sample_params.to("cpu").numpy()
with open("X_valid.npy", "wb") as f:
    np.save(f, X_valid)

In [17]:
with open("Y_valid.npy", "wb") as f:
    np.save(f, torch.cat(list(results), dim=0).numpy())